In [1]:
import sys
import os
path_pipest = os.path.abspath('./')
n=0
while (not os.path.basename(path_pipest)=='pipest') and (n<4):
    path_pipest=os.path.dirname(path_pipest)
    n+=1 
if not os.path.basename(path_pipest)=='pipest':
    print("path_pipest not found. Instead: {}".format(path_pipest))
    raise ValueError("path_pipest not found.")
path_models=path_pipest+'/models'    
path_sdhawkes=path_pipest+'/sdhawkes'
path_modelling = path_sdhawkes+'/modelling'
path_resources = path_sdhawkes+'/resources'
path_lobster=path_pipest+'/lobster_for_sdhawkes'
path_lobster_data=path_lobster+'/data'
path_lobster_pyscripts=path_lobster+'/py_scripts'
path_tests = path_pipest+'/tests'
path_saved_tests = path_tests+'/saved_tests'
path_perfmeas=path_tests+'/performance_measurements'
sys.path.append(path_modelling)
sys.path.append(path_resources)
sys.path.append(path_perfmeas)
import numpy as np
import pandas as pd
import pickle
import datetime
import time
import datetime
import timeit
import model as sd_hawkes_model
import plot_tools

openmp.omp_get_max_threads(): 4


In [2]:
import matplotlib.pyplot as plt

In [3]:
symbol="INTC"
date="2019-01-23"
time_window="37800-41400"

In [4]:
with open(path_models+"/{}/{}_{}/{}_sdhawkes_{}_{}"
          .format(symbol, symbol, date, symbol, date, time_window), 'rb') as source:
    calmodel=pickle.load(source)

AttributeError: Can't get attribute 'RejectSampling' on <module 'lob_model' from '/home/claudio/pipest/pipest/sdhawkes/modelling/lob_model.cpython-37m-x86_64-linux-gnu.so'>

In [ ]:
n_states=[3,5] 
n_events = 4  # number of event types, $d_E$
n_levels = 2
upto_level = 2
time_start=0.0
time_end=time_start+1.0*60 # one minute
model = sd_hawkes_model.SDHawkes(
    number_of_event_types=n_events, list_of_n_states=n_states,
    number_of_lob_levels=n_levels, volume_imbalance_upto_level=upto_level)

In [ ]:
d_E=model.number_of_event_types
d_S=model.number_of_states
print("d_E={}; d_S={}".format(d_E,d_S))

Assign arbitrary values to the Hawkes parameters $\nu_{e}$, $\alpha_{e',x',e}$ and $\beta_{e',x',e}$

In [ ]:
nus = np.array([0.0,0.0,1.0,1.0],dtype=np.float)
alphas = np.maximum(
    np.concatenate(
        [0.2*np.ones((d_E,d_S,d_E//2),dtype=np.float),np.zeros((d_E,d_S,d_E-d_E//2),dtype=np.float)],
        axis=2),
    np.random.uniform(-0.5,0.5,size=(d_E, d_S, d_E))
)
betas = np.exp(np.minimum(2.0,np.random.uniform(0.0001,4.0,size=(d_E, d_S, d_E))))
model.set_hawkes_parameters(nus,alphas,betas)

Assign arbitrary values to the transition matrices $\phi_{e}$, for $e=0,\dots,d_E -1$

In [ ]:
phis = model.state_enc.generate_random_transition_prob(n_events=d_E)
model.set_transition_probabilities(phis)

Assign arbitrary values to the Dirichlet parameters $\gamma_{x}$ for $x = 0,\dots,d_{S}-1$

In [ ]:
gammas = np.random.uniform(low=1.5, high = 5.6,size=(d_S,2*n_levels))
model.set_dirichlet_parameters(gammas)

In [ ]:
model.simulate(time_start, time_end,
               max_number_of_events=10000,
               add_initial_cond=True,store_results=True, report_full_volumes=False)

In [ ]:
model.store_history_of_intensities(density_of_eval_points=5000)

In [ ]:
print(model.simulated_2Dstates.shape)
print(model.simulated_times.shape)

In [ ]:
model.plot_events_and_states(t_0=28.5,t_1=55.0)

In [ ]:
model.plot_events_and_states()

In [ ]:
model.plot_intensities()

In [ ]:
initial_condition_times=np.array(model.simulated_times,copy=True)
initial_condition_events=np.array(model.simulated_events,copy=True)
initial_condition_states=np.array(model.simulated_states,copy=True)
initial_condition_volumes=np.array(model.simulated_volume,copy=True)

In [ ]:
initial_inventory=10
liquidator_base_rate=1.0
type_of_liquid = 'constant_intensity' #constant_intensity or with_the_market or against_the_market
liquidation_control_type='fraction_of_bid_side' # fraction_of_inventory or fraction_of_bid_side
liquidator_control=0.2
time_start=float(initial_condition_times[-1])
time_end=time_start+0.25*60*60

In [ ]:
model.setup_liquidator(initial_inventory=initial_inventory,
                           time_start=time_start,
                           liquidator_base_rate=liquidator_base_rate,
                           type_of_liquid=type_of_liquid,
                           liquidator_control=liquidator_control)
initial_condition_events+=1

In [ ]:
model.liquidator.print_info()

In [ ]:
model.simulate_liquidation(
    time_end,
    initial_condition_events=initial_condition_events,
    initial_condition_states=initial_condition_states,
    initial_condition_times=initial_condition_times,
    initial_condition_volumes=initial_condition_volumes,
    verbose=True,
    report_history_of_intensities = False,
    store_results=True
)

In [ ]:
model.liquidator.print_info()

In [ ]:
model.make_start_liquid_origin_of_times(delete_negative_times=False)

In [ ]:
model.create_impact_profile(delete_negative_times=False,
                            produce_weakly_defl_pp=False,
                            maxiter=10, num_init_guesses=1)

In [ ]:
model.liquidator.impact.store_bm_impact()

In [ ]:
model.store_price_trajectory()

In [ ]:
plt.plot(model.simulated_price[:,0], model.simulated_price[:,1])

In [ ]:
model.store_history_of_intensities()

In [ ]:
model.plot_bm_impact_profile(time_start=-5.0, time_end=80.0)

In [ ]:
model.plot_events_and_states(t_0=-50.5,t_1=105.0, first_event_index=0)